## Problem 3: How many people live in the dominance area of each shopping center? 

Find out how many people live under the dominance area of each shopping center. You should be able to do this based on the output from problem 2, lesson materials from weeks 2, 3 and 4 and these additional hints:

- You can read in the population data from the HSY WFS service: https://kartta.hsy.fi/geoserver/wfs (check more hints from the lesson materials)
- Aggregate your dominance areas from problem 2 into a unified geometries using [`dissolve()`](http://geopandas.org/aggregation_with_dissolve.html#dissolve-example) -function in geopandas before joining with the population data.
- Remember to check the crs of the input data.
- Join information between the population grid and the dominance areas -layer using `intersect` as the condition in the spatial join.

You can freely organize your code into the code cells below.



In [2]:
import geopandas as gpd
import requests
import json

# Read in the population data from WFS using requests
url = 'https://kartta.hsy.fi/geoserver/wfs'
params = dict(service='WFS', 
              version='2.0.0', 
              request='GetFeature', 
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018', 
              outputFormat='json')

r = requests.get(url, params=params)

# We only need the population count, i.e., asukkaita, and the geometry
pop = gpd.GeoDataFrame.from_features(json.loads(r.content))[['asukkaita', 'geometry']]
pop.head()

,asukkaita,geometry
0,9,"POLYGON ((25472499.995 6689749.005, 25472499.9..."
1,5,"POLYGON ((25472499.995 6685998.998, 25472499.9..."
2,8,"POLYGON ((25472499.995 6684249.004, 25472499.9..."
3,5,"POLYGON ((25472499.995 6683999.005, 25472499.9..."
4,11,"POLYGON ((25472499.995 6682998.998, 25472499.9..."


In [4]:
# Get our dominance area data and dissolve it into one shapely object per area
# We only need to dominant_service and geometry columns
grid = gpd.read_file('data/travel_time_grid.geojson', driver='GeoJSON')[['dominant_service', \
                                                                         'geometry']]
grid = grid.dissolve(by='dominant_service').reset_index()
grid.head()

,dominant_service,geometry
0,pt_r_t_Dixi,"MULTIPOLYGON (((384500.000 6675750.000, 384250..."
1,pt_r_t_Forum,"MULTIPOLYGON (((377750.000 6676750.000, 378000..."
2,pt_r_t_IsoOmena,"MULTIPOLYGON (((363250.000 6676750.000, 363000..."
3,pt_r_t_Itis,"MULTIPOLYGON (((389750.000 6672500.000, 389750..."
4,pt_r_t_Jumbo,"MULTIPOLYGON (((375250.000 6684750.000, 375250..."


In [5]:
# check the crs values
print(f'pop crs: {pop.crs}')
print(f'grid crs: {grid.crs}')

pop crs: None
grid crs: epsg:3067


In [7]:
# assign the population CRS
from pyproj import CRS
pop.crs = CRS.from_epsg(3879)
print(f'pop crs: {pop.crs}')

# reasign the grid to match the population crs
grid.to_crs(epsg=3879, inplace=True)
print(f'grid crs: {grid.crs}')

pop crs: epsg:3879
grid crs: epsg:3879


In [11]:
# Join the data and sum the population in the dominant service areas
joined_data = gpd.sjoin(grid, pop, how='left', predicate='intersects')
joined_data_sums = joined_data[['dominant_service', 'asukkaita']].groupby('dominant_service').sum().reset_index()

In [14]:
joined_data

,dominant_service,geometry,index_right,asukkaita
0,pt_r_t_Dixi,"MULTIPOLYGON (((25495359.278 6676600.761, 2549...",3753,1304
0,pt_r_t_Dixi,"MULTIPOLYGON (((25495359.278 6676600.761, 2549...",3855,108
0,pt_r_t_Dixi,"MULTIPOLYGON (((25495359.278 6676600.761, 2549...",3752,10
0,pt_r_t_Dixi,"MULTIPOLYGON (((25495359.278 6676600.761, 2549...",3854,267
0,pt_r_t_Dixi,"MULTIPOLYGON (((25495359.278 6676600.761, 2549...",3751,103
...,...,...,...,...
6,pt_r_t_Ruoholahti,"MULTIPOLYGON (((25487474.735 6672609.299, 2548...",2459,772
6,pt_r_t_Ruoholahti,"MULTIPOLYGON (((25487474.735 6672609.299, 2548...",2411,491
6,pt_r_t_Ruoholahti,"MULTIPOLYGON (((25487474.735 6672609.299, 2548...",2410,1028
6,pt_r_t_Ruoholahti,"MULTIPOLYGON (((25487474.735 6672609.299, 2548...",2458,310


In [12]:
joined_data_sums

,dominant_service,asukkaita
0,pt_r_t_Dixi,223770
1,pt_r_t_Forum,259697
2,pt_r_t_IsoOmena,201205
3,pt_r_t_Itis,212502
4,pt_r_t_Jumbo,79573
5,pt_r_t_Myyrmanni,231630
6,pt_r_t_Ruoholahti,87856


Remember to print the answers in this notebook :)

### Extra bonus task

Repeat problem 2 and 3 for car accessibility! No extra points available for this, but you can for example start thinking if you want to make the final assignment related to this topic :)